In [1]:
from typing import Tuple
import re
import xml.etree.ElementTree as etree
from collections import defaultdict, OrderedDict
from functools import reduce

In [2]:
import numpy as np
import pandas as pd

## Read wikipedia dump

In [11]:
root = etree.parse('../data/kowiki.xml').getroot()

In [12]:
prefix = '{http://www.mediawiki.org/xml/export-0.10/}'
keywords = ['title', 'id', 'revision/text']

In [13]:
context = defaultdict(dict)

In [ ]:
def parse_xml(node, key):
    return node.find(f''{prefix}{key})

In [14]:
for page in root[1:]:
    data = {keyword: reduce(parse_xml, [page, *keyword.split('/')]).text for keyword in keywords}
    context[data['title']] = data

## Parser

In [118]:
regex_document = re.compile('\[\[(.[^:]+?)\]\]')
regex_table = re.compile('\{\{((.|\n)*)\}\}')
regex_table_column = re.compile('\|(.+?)\=([^\|]*)')
regex_redirect = re.compile('#넘겨주기 \[\[(.+?)\]\]')
regex_bracket = re.compile('\((.+?)\)')

regex_tags = OrderedDict({
    'regex_del': (r'\1', re.compile('~~(.+?)~~')),
    'regex_dash': (r'\1', re.compile('--(.+?)--')),
    'regex_link': (r'\1', re.compile('\[\[(.[^:\[\]]+?)\|(.[^:\[\]]+?)\]\]')),
    'regex_attach': ('attach', re.compile('\[\[파일:(.+?)\]\]')),
    'regex_document': (r'\1', re.compile('\[\[(.[^:]+?)\]\]')),
    'regex_tag': ('', re.compile('\<(.+?)\>')),
    'regex_comment': ('', re.compile('\[\*(.+?)\]')),
})

In [49]:
def parse(text: str) -> str:
    def _parse(text: str, target: str, tag: re.Pattern) -> Tuple[str, int]:
        return tag.subn(target, text)
    
    count = 0
    
    while True:
        for target_tag in regex_tags.values():
            text, count = _parse(text, *target_tag)
            if count:
                break
        if not count:
            break
            
    return text.strip()

## Get drama titles from lists

In [50]:
interested = ["대한민국의 텔레비전 드라마 목록/ㄱ", "대한민국의 텔레비전 드라마 목록/ㄴ", "대한민국의 텔레비전 드라마 목록/ㄷ", "대한민국의 텔레비전 드라마 목록/ㄹ", "대한민국의 텔레비전 드라마 목록/ㅁ", "대한민국의 텔레비전 드라마 목록/ㅂ", "대한민국의 텔레비전 드라마 목록/ㅅ", "대한민국의 텔레비전 드라마 목록/ㅇ", "대한민국의 텔레비전 드라마 목록/ㅈ", "대한민국의 텔레비전 드라마 목록/ㅊ", "대한민국의 텔레비전 드라마 목록/ㅋ", "대한민국의 텔레비전 드라마 목록/ㅌ", "대한민국의 텔레비전 드라마 목록/ㅍ", "대한민국의 텔레비전 드라마 목록/ㅎ", "대한민국의 텔레비전 드라마 목록/알파벳", "대한민국의 텔레비전 드라마 목록/숫자", ]

In [51]:
titles = set()

In [53]:
for inter in interested:
    document = context[inter]['revision/text']
    matches = regex_document.findall(document)
    
    for match in matches:
        name, *_ = match.split('|')
        name, *_ = name.split('#')
        if name != '대한민국의 텔레비전 드라마 목록':
            titles.add(name)

## Get metadata from document

In [56]:
columns = {
    '방송 기간': ['방송기간', '방송 기간'],
    '방송 횟수': ['횟수', '방송 횟수'],
    '장르': ['장르'],
    '채널': ['채널', '방송사', '방송 채널', '방송채널'],
    '제작사': ['제작사', '제작자', '제작'],
    '극본': ['극본', '대본', '각본'],
    '출연자': ['출연자', '출연', '출연진'],
}

In [145]:
data = defaultdict(dict)
notexists = []
notparser = []

In [146]:
for title in titles:
    try:
        if '(드라마)' not in title and f'{title} (드라마)' in context:
            document, redirect = context[f'{title} (드라마)']['revision/text'], True
        else:
            document, redirect = context[title]['revision/text'], True
        while redirect:
            document, redirect = regex_redirect.subn(r'\1', document)
            if redirect:
                document = context[document]['revision/text']
    except KeyError:
        notexists.append(title)
    
    d = defaultdict(str)    
    for table, *_ in regex_table.findall(document):
        for key, value in regex_table_column.findall(table):
            key = next((ckey for ckey, cvalues in columns.items() if any(cvalue in key for cvalue in cvalues)), False)
            if key:
                key = key.strip()
                d[key] = f'{d[key]} {value}'
    if not d:
        notparser.append(title)
    else:
        data[title] = d

In [147]:
print(f'{len(notparser)} pages are not parsable')
print(f'{len(notexists)} pages are not exists')

11 pages are not parsable
16 pages are not exists


## Create table from data

In [251]:
table_columns = list(columns.keys())
table = np.empty((0, len(table_columns) + 1))

for title, values in data.items():
    table = np.vstack((table, np.array([
        regex_bracket.sub('', title).replace(' ', '').strip(), *tuple(map(parse, map(lambda c: values[c], table_columns)))
    ])))

In [258]:
df = pd.DataFrame(table)
df.columns = ['제목', *table_columns]

### Parse datetime

In [259]:
regex_date = re.compile('(.+?)년(.*)월(.*)일')

In [260]:
date_start = []
for index, date in enumerate(df['방송 기간']):
    ds, *de = map(regex_date.findall, map(str.strip, date.split('~' if '~' in date else '-')))
    ds, *_ = ds or ['unknown']
    try:
        date_start.append(pd.datetime(*tuple(map(int, ds))))
    except (ValueError, TypeError):
        date_start.append('unknown')
    
assert len(date_start) == np.size(df, 0)

In [261]:
df['방송 시작'] = pd.Series(date_start)

## Show Dataframe

In [262]:
print(df.shape)
df.head()

(1762, 9)


,제목,방송 기간,방송 횟수,장르,채널,제작사,극본,출연자,방송 시작
0,언터처블,2017년 11월 24일 ~ 2018년 1월 20일,16부작,드라마,JTBC,"손기원, 박준서\n 김종학 프로덕션, [[드라마하우스앤드제이콘텐트허브",최진원,"진구, [[김성균 (배우)",2017-11-24 00:00:00
1,마을-아치아라의비밀,2015년 10월 7일 ~ 2015년 12월 3일,16부작,미스터리 스릴러 드라마,[[SBS (대한민국의 방송사),김동래\n 래몽래인,도현정,"문근영, 육성재, 온주완, [[신은경 (배우)",2015-10-07 00:00:00
2,사랑과결혼,1995년 4월 22일 ~ 1995년 10월 15일,48부작,[[텔레비전 드라마,[[문화방송,,[[김지수 (작가),"김희애, 김혜수, 이영애, 손창민 외",1995-04-22 00:00:00
3,야호,1987년 3월 9일 ~ 1987년 3월 31일,8부작,[[텔레비전 드라마,[[문화방송,,정하연,"한애경, [[김동현 (1950년)",1987-03-09 00:00:00
4,별도달도따줄게,2012년 5월 7일 ~ 2012년 11월 2일,129부작,[[텔레비전 드라마 드라마 삽입곡,KBS 1TV,"김선진, 김영일, 이봉호, 문종식",홍영희,"서지혜, 조동혁, 고세원, 문보령 외",2012-05-07 00:00:00


In [263]:
df.to_csv('../results/wikipedia.csv', index=None)